In [1]:
import boto3
import time
import urllib
import json
    
transcribe_client = boto3.client('transcribe')

def transcribe_file(job_name, file_uri, transcribe_client):
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': file_uri},
        MediaFormat='mp3',
        LanguageCode='en-US'
    )

    max_tries = 60
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job['TranscriptionJob']['TranscriptionJobStatus']
        if job_status in ['COMPLETED', 'FAILED']:
            print(f"Job {job_name} is {job_status}.")
            if job_status == 'COMPLETED':
                response = urllib.request.urlopen(job['TranscriptionJob']['Transcript']['TranscriptFileUri'])
                data = json.loads(response.read())
                text = data['results']['transcripts'][0]['transcript']
                print("========== below is output of speech-to-text ========================")
                print(text)
                print("=====================================================================")
            break
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)


def main():
    file_uri = 's3://vivekp/English in a Minute Grow Like a Weed.mp4'
    transcribe_file('Example-job', file_uri, transcribe_client)


if __name__ == '__main__':
    main()

Waiting for Example-job. Current status is IN_PROGRESS.
Job Example-job is COMPLETED.
========== below is output of speech-to-text ========================
Welcome to English in a minute. If you've ever gardened, you know, weeds grow super fast. They can take over a garden overnight. But what does grow like a weed mean in other situations? Hey, Dan. Do you want to see pictures from my trip home? Oh, jeez. I would love to. Uh, but great. This is me with my little cousin, Arlo. Little. He's taller than you. I know he is growing like a weed. He's only 10. Does he have a beard to grow like a weed just means to grow very quickly. We mostly use this expression when talking about how fast Children grow. And that's English in a minute.
